In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score

In [62]:
train = pd.read_csv('../datasets/Transformed Training data.csv')
price = pd.read_csv('../datasets/Selected train data.csv')
test = pd.read_csv('../datasets/test.csv')

In [63]:
X = train
y = price['SalePrice']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [65]:
lr = LinearRegression()

In [66]:
lr.fit(X_train, y_train)

LinearRegression()

In [67]:
y_predict = lr.predict(X_test)

In [68]:
lr.score(X_train, y_train)

0.8737248118097686

In [69]:
metrics.mean_absolute_error(y_test, y_predict)

20440.790833032705

In [70]:
preds = lr.predict(test)
preds.shape

(878,)

In [71]:
test

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,Id
0,6.0,1910.0,440.0,0.0,1020.0,1928.0,908.0,1020.0,36.0,11460.0,...,1966560.0,926160.0,1040400.0,3717184.0,1750624.0,1966560.0,824464.0,926160.0,1040400.0,109
1,5.0,1977.0,580.0,0.0,1967.0,1967.0,1967.0,0.0,25.0,9885.0,...,3869089.0,3869089.0,0.0,3869089.0,3869089.0,0.0,3869089.0,0.0,0.0,544
2,7.0,2006.0,426.0,2.0,654.0,1496.0,664.0,832.0,49.0,14042.0,...,978384.0,434256.0,544128.0,2238016.0,993344.0,1244672.0,440896.0,552448.0,692224.0,153
3,5.0,1923.0,480.0,0.0,968.0,968.0,968.0,0.0,25.0,9615.0,...,937024.0,937024.0,0.0,937024.0,937024.0,0.0,937024.0,0.0,0.0,318
4,6.0,1963.0,514.0,1.0,1394.0,1394.0,1394.0,0.0,36.0,11778.0,...,1943236.0,1943236.0,0.0,1943236.0,1943236.0,0.0,1943236.0,0.0,0.0,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,6.0,1974.0,488.0,2.0,1084.0,1877.0,1084.0,793.0,36.0,11844.0,...,2034668.0,1175056.0,859612.0,3523129.0,2034668.0,1488461.0,1175056.0,859612.0,628849.0,2238
874,6.0,1966.0,480.0,2.0,1104.0,1988.0,1104.0,884.0,36.0,11796.0,...,2194752.0,1218816.0,975936.0,3952144.0,2194752.0,1757392.0,1218816.0,975936.0,781456.0,982
875,5.0,1968.0,322.0,0.0,952.0,1211.0,1211.0,0.0,25.0,9840.0,...,1152872.0,1152872.0,0.0,1466521.0,1466521.0,0.0,1466521.0,0.0,0.0,1265
876,4.0,1971.0,528.0,0.0,864.0,864.0,864.0,0.0,16.0,7884.0,...,746496.0,746496.0,0.0,746496.0,746496.0,0.0,746496.0,0.0,0.0,559


In [72]:
test['SalePrice'] = preds

In [73]:
#test['0'].astype(int)
#test.rename(columns={'0':'Id'}, inplace=True)
lr_submission = test[['Id', 'SalePrice']].set_index('Id')

In [74]:
lr_submission['SalePrice'] = lr_submission['SalePrice'].abs()


In [75]:
lr_submission

,SalePrice
Id,
109,3.233210e+09
544,2.370846e+04
153,2.142952e+09
318,1.549138e+05
255,1.305248e+05
...,...
2238,1.985419e+09
982,2.459133e+09
1265,1.068110e+05


In [76]:
lr_submission.to_csv('../datasets/baseline.csv')